In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
from scipy.stats import poisson
from scipy.stats import nbinom

# INPUT

In [ ]:
# demographics: Ile-de-France

N = 12278210  #total population 

#population per age class
N_c = round(0.1447*N)  #children   [0,11) years
N_t = round(0.1113*N)  #teens      [11,19) years
N_a = round(0.59*N)    #adults     [19,65) years
N_s = round(0.154*N)   #over 65+   over 65+ 

ages = 4 #number of age classes

In [ ]:
# epidemiological parameters

incubation = 3.7       # incubation period (days) 
prodromic = 1.5        # prodromic phase (days)
infection = 2.3        # infectious period (days)

delta_t = 1   # one timestep = 1 day

# compute transition rates
sigma=1./incubation
theta=1./prodromic
gamma=1./infection    

# fraction of asymptomatic
asint = 0.4
p_as = np.array([asint]*ages)

#probability of developing pauci/mild/severe symptoms if not asymptomatic
# explicit value for each age class
ps = np.array([1.,1.,0.2,0.2])  
ms = np.array([0.,0.,0.8-0.04,0.8-0.27])  
ss = np.array([0.,0.,0.04,0.27])  

# fraction entering I_ps, I_ms, I_ss compartments
p_ps = np.multiply(1-p_as, ps)  
p_ms = np.multiply(1-p_as, ms)
p_ss = np.multiply(1-p_as, ss)

#compartments for hospitalitalizations and intensive care units

#probability of entering ICU if developing severe symptoms (I_ss)
p_U = np.array([0., 0., 0.24357, 0.24357])

#probability of entering H (no ICU) if developing severe symptoms (I_ss)
p_H = np.array([0., 0., 1.0-0.24357, 1.0-0.24357])

#transition rate out of ICU
mu_U = np.array([0. ,0. ,1./21.1,1./20.689])

#probabilty of recovery/death if going through ICU
p_R_from_ICU = np.array([0.,0.,0.76, 0.54])
p_D_from_ICU = np.array([0.,0.,1.0-0.76, 1.0-0.54])

#transition rate out of H
delta_H_R = np.array([0, 0, 0.083, 0.033])    #H->R
delta_H_D = np.array([0, 0, 0.0031,0.0155])   #H->D

# relative susceptibility
susc = np.array([0.5,0.5,1,1])      # explicit value for each age class

In [ ]:
# relative infectiousness
r_children = float(input('Choose relative infectiousness for younger children \n(Values used in the manuscript: 0.55, 0.33, 0.35, 0.1) \n'))
r = np.array([r_children,0.55,0.55,0.55]) # explicit value for each age class

In [ ]:
# contact matrices

#function to read matrix
def read(df):
    C = []
    for i in range(ages*ages):
        c = df[0][i]
        C.append(c)
    C = np.array(C)
    C = np.reshape(C, (ages,ages))
    return C 

#function to compute contact matrices w/ testing
def extract_matrix(matrix, x_test):
    m_p = read(matrix) # for cases in the prodromic phase
    m_t = x_test*read(matrix)*0.1 + (1-x_test)*read(matrix)  # apply testing for asymptomatic, pauci-symptomatic and mild cases
    m_ss = x_test*read(matrix)*0.1 + (1-x_test)*read(matrix)*0.25  # apply testing for severe cases

    return m_p,m_t,m_ss

# TRANSMISSION MODEL

In [ ]:
def seir(u, parms, t, intervention, LD, scale_LD, 
          BS_matrix, LD_matrix,
          week_matrix, summer_matrix):
    
    """
    Simulation one time step (day) of the compartmental, age-stratified model.
    INPUT
    u: dict containing the number of individuals (incidence and prevalence) for each compartment in the previous timestep
    parms: list of epidemiological parameters
    t: current timestep
    intervention: boolean indicating if an intervention measure (e.g. lockdown) is put in place
    LD: list containing the timesteps corresponding to start and end of lockdown
    scale_LD: scaling factor of the pre-LD transmission rate, fitted during lockdown 
    BS_matrix: contact matrix in the pre-lockdown phase
    LD_matrix: contact matrix during the lockdown
    week_matrix: weekly contact matrix in the post-lockdown phase, prior to summer holidays    
    summer_matrix: contact matrix during summer holidays
    RETURNS: updated version of dict u
    """

    #quantities of interest
    S = u['S']
    new_E = u['Y_E']
    E = u['E']
    new_I_p = u['Y_I_p']  
    I_p = u['I_p'] 
    new_I_as = u['Y_I_as'] 
    I_as = u['I_as'] 
    new_I_ps = u['Y_I_ps'] 
    I_ps = u['I_ps'] 
    new_I_ms = u['Y_I_ms'] 
    I_ms = u['I_ms'] 
    new_I_ss = u['Y_I_ss'] 
    I_ss = u['I_ss'] 
    new_H = u['Y_H'] 
    H = u['H'] 
    new_U = u['Y_U']
    U = u['U'] 
    new_R = u['Y_R'] 
    R = u['R'] 
    new_D = u['Y_D'] 
    D = u['D'] 
    N_tot = u['N_tot'] 

    # epidemiological parameters
    bet, sigm, thet, gamm, N, dt = parms
    
    # contact matrices
    
    # pre-lockdown
    BS_p, BS_t, BS_ss = extract_matrix(BS_matrix,x_test=0) 
    C_p = BS_p 
    C_t = BS_t
    C_ss = BS_ss     
    
    #lockdown
    LD_p, LD_t, LD_ss = extract_matrix(LD_matrix,x_test=0)
    
    if intervention:
        for i in range(len(LD)):
            LD_start = LD[i][0]
            LD_end = LD[i][1]
            # during lockdown
            if (t >= LD_start) and (t < LD_end): 
                C_p = LD_p*scale_LD
                C_t = LD_t*scale_LD
                C_ss = LD_ss*scale_LD
        for i in range(8):
            if (t >= (m_11 + i*7)) and (t < (m_11+(i+1)*7)):
                C_p, C_t, C_ss = extract_matrix(week_matrix[i],x_test=0.5)
        if (t >= start_summ) and (t < end_summ):
            C_p, C_t, C_ss = extract_matrix(summer_matrix,x_test=0.5)
                    
    # force of infection
    lambd = []
    for age in range(ages):
        l = 0 
        for age2 in range(ages):
            l += susc[age]*r[age2]*bet*C_p[age,age2]*I_p[age2]/N
            l += susc[age]*r[age2]*bet*C_t[age,age2]*I_as[age2]/N
            l += susc[age]*r[age2]*bet*C_t[age,age2]*I_ps[age2]/N
            l += susc[age]*bet*C_t[age,age2]*I_ms[age2]/N
            l += susc[age]*bet*C_ss[age,age2]*I_ss[age2]/N
        lambd.append(l)
    lambd = np.array(lambd)

    # compute transition probabilities for entering in each comparment
    in_E = 1 - np.exp(-lambd*dt)
    in_I_p = np.array([sigm*dt]*ages)
    in_I = np.array([thet*dt]*ages)
    in_R = np.array([gamm*dt]*ages)
    in_H = np.array([gamm*dt]*ages)
    
    H_to_R = delta_H_R*dt 
    H_to_D = delta_H_D*dt
    U_to = mu_U*dt
    
    # transition events (sampled by binomial or multinomial distributions with corresponding transition probabilities  
    for age in range(ages):
        #transitions
        new_E[age] = np.random.binomial(S[age],in_E[age])
        new_I_p[age] = np.random.binomial(E[age],in_I_p[age])
        
        trans_I = np.array([p_as[age]*in_I[age], p_ps[age]*in_I[age], p_ms[age]*in_I[age], p_ss[age]*in_I[age], 1-in_I[age]])
        new_I_as[age], new_I_ps[age], new_I_ms[age], new_I_ss[age], res = np.random.multinomial(I_p[age],trans_I)
        
        recovery_as = np.random.binomial(I_as[age],in_R[age])
        recovery_ps = np.random.binomial(I_ps[age],in_R[age])
        recovery_ms = np.random.binomial(I_ms[age],in_R[age])
        
        trans_H = np.array([p_H[age]*in_H[age], p_U[age]*in_H[age], 1-in_H[age]])
        new_H[age], new_U[age], res = np.random.multinomial(I_ss[age],trans_H)   
        
        exit_H = np.array([H_to_R[age], H_to_D[age], 1- H_to_R[age]- H_to_D[age]])
        recovery_h, death_h, res = np.random.multinomial(H[age], exit_H)

        exit_ICU = np.array([
            p_R_from_ICU[age]*U_to[age], p_D_from_ICU[age]*U_to[age], 
            1-p_R_from_ICU[age]*U_to[age]-p_D_from_ICU[age]*U_to[age]])
        recovery_icu, death_icu, res = np.random.multinomial(U[age], exit_ICU)
        
        new_R[age] = recovery_as + recovery_ps + recovery_ms + recovery_h + recovery_icu 
        new_D[age] = death_icu + death_h
        
        #update compartments
        S[age] = S[age] - new_E[age] 
        E[age] = E[age] + new_E[age] - new_I_p[age]
        I_p[age] = I_p[age] + new_I_p[age] - new_I_as[age] - new_I_ps[age] - new_I_ms[age] - new_I_ss[age]
        I_as[age] = I_as[age] + new_I_as[age] - recovery_as
        I_ps[age] = I_ps[age] + new_I_ps[age] - recovery_ps
        I_ms[age] = I_ms[age] + new_I_ms[age] - recovery_ms
        I_ss[age] = I_ss[age] + new_I_ss[age] - new_H[age] - new_U[age]
        H[age] = H[age] + new_H[age] - recovery_h - death_h
        U[age] = U[age] + new_U[age] - recovery_icu -death_icu
        R[age] = R[age] + new_R[age]
        D[age] = D[age] + new_D[age]
        N_tot[age] = S[age]+E[age]+I_p[age]+I_as[age]+I_ps[age]+I_ms[age]+I_ss[age]+H[age]+U[age]+R[age]+D[age]

   
    return  {'t':t, 'S': S, 'E':E, 'I_p':I_p, 
             'I_as':I_as,'I_ps':I_ps,'I_ms':I_ms,'I_ss':I_ss,
             'H':H,'U':U,'R':R,'D':D,
             'Y_E':new_E,'Y_I_p':new_I_p,'Y_I_as':new_I_as,
             'Y_I_ps':new_I_ps, 'Y_I_ms':new_I_ms, 'Y_I_ss':new_I_ss, 
             'Y_H':new_H,'Y_U':new_U,'Y_R':new_R,'Y_D':new_D,'N_tot':N_tot}

In [ ]:
def simulate(intervention, LD, scale_LD, BS_matrix, LD_matrix, week_matrix, summer_matrix):
    
    """
    Simulation of one single stochastic run of the compartmental, age-stratified model.
    INPUT
    intervention: boolean indicating if an intervention measure (e.g. lockdown) is put in place
    LD: list containing the timesteps corresponding to start and end of lockdown
    scale_LD: scaling factor of the pre-LD transmission rate, fitted during lockdown 
    BS_matrix: contact matrix in the pre-lockdown phase
    LD_matrix: contact matrix during the lockdown
    week_matrix: weekly contact matrix in the post-lockdown phase, prior to summer holidays    
    summer_matrix: contact matrix during summer holidays
    RETURNS: dict containing the number of individuals for each compartment, for each timestep
    """
    
    parms = [beta, sigma, theta, gamma, N, delta_t]
    
    tf = t_stop
    
    t = np.arange(tf)
    S = np.zeros((tf,ages))
    E = np.zeros((tf,ages))
    I_p = np.zeros((tf,ages))   
    I_as = np.zeros((tf,ages))
    I_ps = np.zeros((tf,ages)) 
    I_ms = np.zeros((tf,ages))
    I_ss = np.zeros((tf,ages))
    H = np.zeros((tf,ages))
    U = np.zeros((tf,ages))
    D = np.zeros((tf,ages))
    R = np.zeros((tf,ages))
    Y_E = np.zeros((tf,ages))
    Y_I_p = np.zeros((tf,ages))
    Y_I_as = np.zeros((tf,ages))
    Y_I_ps = np.zeros((tf,ages))
    Y_I_ms = np.zeros((tf,ages))
    Y_I_ss = np.zeros((tf,ages))
    Y_H = np.zeros((tf,ages))
    Y_U = np.zeros((tf,ages)) 
    Y_R = np.zeros((tf,ages))
    Y_D = np.zeros((tf,ages))
    N_tot = np.zeros((tf,ages))
    
    result = {'t':t, 'S': S, 'E':E, 'I_p':I_p, 'I_as':I_as,'I_ps':I_ps,'I_ms':I_ms,'I_ss':I_ss,
            'H':H,'U':U,'R':R,'D':D,'Y_E':Y_E,'Y_I_p':Y_I_p,'Y_I_as':Y_I_as,
            'Y_I_ps':Y_I_ps, 'Y_I_ms':Y_I_ms, 'Y_I_ss':Y_I_ss,'Y_H':Y_H,'Y_U':Y_U,'Y_R':Y_R,'Y_D':Y_D,'N_tot':N_tot}

    #initial condition
    
    u = {'t':0, 'S': [N_c,N_t,N_a-I_seed,N_s], 'E':[0]*ages, 
         'I_p':[0,0,I_seed,0], 'I_as':[0]*ages,'I_ps':[0]*ages,'I_ms':[0]*ages,'I_ss':[0]*ages,
        'H':[0]*ages,'U':[0]*ages,'R':[0]*ages,'D':[0]*ages,'Y_E':[0]*ages,'Y_I_p':[0]*ages,'Y_I_as':[0]*ages,
            'Y_I_ps':[0]*ages, 'Y_I_ms':[0]*ages, 'Y_I_ss':[0]*ages, 
         'Y_H':[0]*ages,'Y_U':[0]*ages,'Y_R':[0]*ages,'Y_D':[0]*ages,'N_tot':[N_c,N_t,N_a,N_s]}   

    #save initial condition
    for c in result.keys():
        result[c][0] = u[c]
    
    for j in range(1,tf):
        u = seir(u,parms,t[j],intervention, LD, scale_LD, BS_matrix, LD_matrix, week_matrix, summer_matrix)
        for c in result.keys():
            result[c][j] = u[c]
            
    return result 

In [ ]:
def run_simulation(intervention = False, LD = [[0,0]], scale_LD=0, week_matrix=0, summer_matrix=0):
    
    """
    Simulation of multiple stochastic runs of the compartmental, age-stratified model.
    INPUT
    intervention: boolean indicating if an intervention measure (e.g. lockdown) is put in place
    LD: list containing the timesteps corresponding to start and end of lockdown
    scale_LD: scaling factor of the pre-LD transmission rate, fitted during lockdown 
    RETURNS: dict containing the number of individuals for compartments of interest (e.g. incidence in the hospital compartment), for each timestep, for each run
    """
                   
    #contact matrices 
                   
    #pre-lockdown
    BS_matrix = pd.read_table('./input/matrices/baseline.txt', header = None, sep=' ')
 
    #lockdown
    LD_matrix = pd.read_table('./input/matrices/LD.txt', header = None, sep=' ')
    
    out_Y_HU = pd.DataFrame(np.arange(t_stop))
    out_U = pd.DataFrame(np.arange(t_stop))
    out_U_prev = pd.DataFrame(np.arange(t_stop))
    out_Y_cl = pd.DataFrame(np.arange(t_stop))
    
    #for each run
    for n in range(n_runs):
        out = simulate(intervention, LD, scale_LD, BS_matrix, LD_matrix, week_matrix, summer_matrix)
        # extract quantities of interest, e.g. number of recovered or incidence
        # save the timeseries in the column of a dataframe
        
        out_Y_HU[n] = pd.DataFrame(out['Y_H']).sum(axis=1)+pd.DataFrame(out['Y_U']).sum(axis=1)
        out_U[n] = pd.DataFrame(out['Y_U']).sum(axis=1)
        out_U_prev[n] = pd.DataFrame(out['U']).sum(axis=1) 
        out_Y_cl[n] = pd.DataFrame(out['Y_I_ms']).sum(axis=1) + pd.DataFrame(out['Y_I_ss']).sum(axis=1)
        
    return {'adm_H':out_Y_HU, 'adm_ICU':out_U, 'prev_ICU':out_U_prev, 'clinical':out_Y_cl} 

In [ ]:
def add_median_CI(DF):
    df = DF.copy()
    df['p1'] = df[[i for i in range(n_runs)]].quantile(0.025, axis=1)
    df['median'] = df[[i for i in range(n_runs)]].median(axis=1)  
    df['p2'] = df[[i for i in range(n_runs)]].quantile(0.975, axis=1)
    return df

# LOAD DATA

In [ ]:
data = pd.read_csv(r'./input/hosp_data.csv')
ICU_adm = data[data['type']=='new_ICU_admissions'].reset_index()

## calibration during lockdown

In [ ]:
I_seed = 50  # initial number of infected (adults in I_p compartment)

In [ ]:
#read results of calibration pre-lockdown

#maximum likelihood estimators obtained according to a given value of the relative infectiousness of younger children
betas = {'0.55': 0.0942, '0.33': 0.0952, '0.25': 0.0954, '0.1': 0.0956}
lags = {'0.55': 19, '0.33': 19, '0.25': 19, '0.1': 19}

beta=betas[str(r_children)] #transmission rate per contact
lag=lags[str(r_children)]  #number of days before March 1

#set calendar
calendar=pd.DataFrame(pd.date_range(dt.date(2020, 3, 1)-dt.timedelta(days=int(lag)), periods=365)) 

# timesteps for lockdown start and end date (March 17 - May 11) and summer holidays 

start_ld = lag + 16 #March 17
start = start_ld + 5
m_11 = start_ld + 55 #May 11

#summer holidays 
start_summ = lag+126 #July 5
end_summ = lag+184   #September 1

t_stop = m_11 #number of (daily) timesteps (run simulation up to May 11)

### fit scaling factor during lockdown

In [ ]:
n_runs = int(input('Choose number of stochatic runs: ')) #500 stochastic runs were used in the paper

In [ ]:
# fit scaling factor

lh = pd.DataFrame({'scale': [],'loglike': []})

scales = np.arange(0.9, 1.2, 0.01)  #explore values of the scaling factor  
for s in scales:
    output = run_simulation(intervention=True,LD=[[start,m_11]],
                         scale_LD=s, week_matrix=0, summer_matrix=0)  
    median_adm_ICU = add_median_CI(output['adm_ICU'])['median']
    p=0
    for t in range(43,43+7*2): # April 13 -> April 26
        #compute log likelihood
        p += poisson.logpmf(ICU_adm['obs'].iloc[t], mu=median_adm_ICU.iloc[t+lag])
    print(s,p)
    lh = lh.append({'scale':s, 'loglike':p}, ignore_index=True)
    
#take value that maximizise the likelihood
scale_best=lh.iloc[lh['loglike'].idxmax()]['scale']
scale_best

# Scenarios of school reopening

In [ ]:
t_stop = end_summ
scale_LD = scale_best

In [ ]:
SC_h = pd.read_table('./input/matrices/SC.txt', header = None, sep=' ')
VC_h = pd.read_table('./input/matrices/vacation.txt', header = None, sep=' ') 

In [ ]:
for teleschool in ['kids_only', 'kids_before_teens', 'kids_and_teens']:                                            
    
    # first set of scenarios: reopening of pre-schools and primary schools only, starting May 11
    if teleschool =='kids_only':
        #school closed for 75% of younger students (attendance allowed 25%)
        w1 = pd.read_table('./input/matrices/TS_75_100.txt', header = None, sep=' ')
        #school closed for 50% of younger students (attendance allowed 50%)
        w2 = pd.read_table('./input/matrices/TS_50_100.txt', header = None, sep=' ')
        #school closed for 25% of younger students (attendance allowed 75%)
        w3 = pd.read_table('./input/matrices/TS_25_100.txt', header = None, sep=' ')
        #schools opened (SO) for younger children only
        w4 = pd.read_table('./input/matrices/SO_kids.txt', header = None, sep=' ')
        
    # second set of scenarios: 
    # reopening of pre-schools and primary schools on May 11, through Progressive (100%) 
    # followed by the reopening of middle and high schools on June 8 through all 4 possible protocols
    elif teleschool =='kids_before_teens':
        #
        w1 = pd.read_table('./input/matrices/TS_75_100.txt', header = None, sep=' ')
        #
        w2 = pd.read_table('./input/matrices/TS_50_100.txt', header = None, sep=' ')
        #
        w3 = pd.read_table('./input/matrices/TS_25_100.txt', header = None, sep=' ')
        #
        w4 = pd.read_table('./input/matrices/SO_kids.txt', header = None, sep=' ')
        #
        w5 = pd.read_table('./input/matrices/TS_0_75.txt', header = None, sep=' ')
        #
        w6 = pd.read_table('./input/matrices/TS_0_50.txt', header = None, sep=' ')
        #
        w7 = pd.read_table('./input/matrices/TS_0_25.txt',header = None, sep=' ')
        #
        w8 = pd.read_table('./input/matrices/SO.txt', header = None, sep=' ')
        
    # third set of scenarios: reopening of all schools on May 11
    elif teleschool =='kids_and_teens':
        #school closed for 75% of students (attendance allowed 25%)
        w1 = pd.read_table('./input/matrices/TS_75_75.txt', header = None, sep=' ')
        #school closed for 50% of students (attendance allowed 50%)
        w2 = pd.read_table('./input/matrices/TS_50_50.txt', header = None, sep=' ')
        #school closed for 25% of students (attendance allowed 75%)
        w3 = pd.read_table('./input/matrices/TS_25_25.txt', header = None, sep=' ')
        #schools open (attendance allowed 100%)
        w4 = pd.read_table('./input/matrices/SO.txt', header = None, sep=' ')

        
    protocols = []
        
    #school closed
    out = run_simulation(intervention=True, LD=[[start,m_11]], scale_LD=scale_LD, 
                         week_matrix = [SC_h for i in range(8)], summer_matrix = SC_h) 
    protocols.append(out)
    
    # reopening protocols

    # Progressive 100%    
    if teleschool!='kids_before_teens':
        out = run_simulation(intervention=True, LD=[[start,m_11]], scale_LD=scale_LD, 
                             week_matrix = [w1,w2,w3,w4,w4,w4,w4,w4], summer_matrix = VC_h)
    else:
        out = run_simulation(intervention=True, LD=[[start,m_11]], scale_LD=scale_LD, 
                             week_matrix = [w1,w2,w3,w4,w5,w6,w7,w8], summer_matrix = VC_h)
    protocols.append(out)

    # Progressive 50%
    if teleschool!='kids_before_teens':
        out = run_simulation(intervention=True, LD=[[start,m_11]], scale_LD=scale_LD, 
                             week_matrix = [w1,w2,w2,w2,w2,w2,w2,w2], summer_matrix = VC_h)
    else:
        out = run_simulation(intervention=True, LD=[[start,m_11]], scale_LD=scale_LD, 
                            week_matrix = [w1,w2,w3,w4,w5,w6,w6,w6], summer_matrix = VC_h)
    protocols.append(out)

    # Prompt 50%
    if teleschool!='kids_before_teens':
        out = run_simulation(intervention=True, LD=[[start,m_11]], scale_LD=scale_LD,
                         week_matrix = [w2 for i in range(8)], summer_matrix = VC_h)
    else:
        out = run_simulation(intervention=True, LD=[[start,m_11]], scale_LD=scale_LD, 
                             week_matrix = [w1,w2,w3,w4,w6,w6,w6,w6], summer_matrix = VC_h)
    protocols.append(out)

    # Prompt 100%
    if teleschool!='kids_before_teens':
        out = run_simulation(intervention=True, LD=[[start,m_11]], scale_LD=scale_LD,
                             week_matrix = [w4 for i in range(8)], summer_matrix = VC_h)
    else: 
        out = run_simulation(intervention=True, LD=[[start,m_11]], scale_LD=scale_LD, 
                             week_matrix = [w1,w2,w3,w4,w8,w8,w8,w8], summer_matrix = VC_h)
    protocols.append(out)

    #########################   
    
    beds = []
    incid = []

    for out in protocols:

        ddf = out['prev_ICU'].copy()
        ddf = add_median_CI(ddf)
        beds.append(ddf)

        ddf = out['clinical'].copy()
        ddf = add_median_CI(ddf)
        incid.append(ddf)

    labels = ['closure','progr_100','progr_50','prompt_50','prompt_100']

    for i in range(len(beds)):
        df = beds[i]
        protocol = labels[i]
        df['date'] = pd.DataFrame(pd.date_range(dt.date(2020, 3, 1)-dt.timedelta(days=int(lag)), periods=len(df)))
        df[['date','p1', 'median', 'p2']].to_csv('./output/r{}_{}_{}_ICUbeds.csv'.format(r_children,teleschool,protocol),index=False)
    for i in range(len(incid)):
        df = incid[i]
        protocol = labels[i]
        df['date'] = pd.DataFrame(pd.date_range(dt.date(2020, 3, 1)-dt.timedelta(days=int(lag)), periods=len(df)))
        df[['date','p1', 'median', 'p2']].to_csv('./output/r{}_{}_{}_new_clinical_cases.csv'.format(r_children,teleschool,protocol), index=False)